In season: 10-01 through 03-31
Off season: 04-01 through 09-30

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import the map to nhl player id data
nhl_id_mapping = pd.read_csv(r'../Data/map_player_nhl.csv')

In [3]:
# Import the trade dates data
trade_dates = pd.read_csv(r'../Data/trade_dates.csv')

In [4]:
# Import the traded players stats from the season before being traded.
traded_players_pre_season = pd.read_csv(r'../Data/traded_skater_platform.csv')

In [5]:
# Import the gen info data
gen_info = pd.read_csv(r'../Data/traded_player_general_info.csv')

In [6]:
# Merge trade dates and players data
first_merge = pd.merge(traded_players_pre_season, nhl_id_mapping, on='playerId')

In [7]:
# Merge last result with trade dates data
second_merge = pd.merge(first_merge, trade_dates, on='tradeId', how='inner')

In [8]:
# Merge last result with the general player info data
third_merge = pd.merge(second_merge, gen_info, on=['playerId','position'], how='inner')

In [9]:
third_merge.shape

(1488, 32)

In [10]:
# Read in contractual data
contract_data = pd.read_csv(r'../Data/traded_contracts.csv')

In [11]:
third_merge.columns

Index(['playerId', 'position', 'positionGeneral_x', 'league', 'leagueLevel',
       'tradeId', 'regularGamesPlayed', 'regularGoals', 'regularAssists',
       'regularPenaltyMinutes', 'postseasonGamesPlayed', 'postseasonGoals',
       'postseasonAssists', 'postseasonPenaltyMinutes', 'regular_hits',
       'regular_blockedShots', 'regular_powerplayGoals', 'regular_shots',
       'regular_atoi', 'regular_pp_atoi', 'regular_pk_atoi', 'nhlPlayerId',
       'trade_date', 'positionGeneral_y', 'height_cm', 'weight_kg',
       'handedness', 'dateOfBirth', 'birthCountry', 'draftYear', 'draftRound',
       'draftOverallPick'],
      dtype='object')

In [12]:
contract_data.columns

Index(['contractId', 'playerId', 'season', 'seasonStart', 'seasonEnd', 'aav',
       'tradeId', 'toTeamId', 'fromTeamId', 'amountOfCapRetained',
       'acquiringCap'],
      dtype='object')

In [13]:
third_merge[['playerId','tradeId']].value_counts()

playerId  tradeId
832445    3063       2
202       2258       1
587498    991        1
592246    737        1
592066    1088       1
                    ..
107608    283        1
107548    224        1
107488    247        1
107087    1146       1
864629    1315       1
Name: count, Length: 1487, dtype: int64

In [14]:
third_merge[third_merge['playerId'] == 832445]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,trade_date,positionGeneral_y,height_cm,weight_kg,handedness,dateOfBirth,birthCountry,draftYear,draftRound,draftOverallPick
86,832445,RW,F,nhl,professional,3063,5,1,1,0,...,2024-07-03,F,190.5,101.0,R,2000-06-07,Russia,2020.0,2.0,61.0
87,832445,RW,F,nhl,professional,3063,5,1,1,0,...,2024-07-03,F,190.5,101.0,R,2000-06-07,Russia,2020.0,2.0,61.0


In [15]:
third_merge.shape

(1488, 32)

In [16]:
# Mix up in these two rows so drop them
third_merge = third_merge[~(third_merge['playerId'] == 832445)]

In [17]:
third_merge.shape

(1486, 32)

In [18]:
# Merge the last merge result with contractual data
fourth_merge = pd.merge(third_merge, contract_data, on=['playerId','tradeId'], how='inner')

In [19]:
fourth_merge.shape

(1476, 41)

In [20]:
# Import data on team ID's
team_id_mapping = pd.read_csv(r'../Data/team_season_mapping.csv')

In [21]:
fourth_merge.columns

Index(['playerId', 'position', 'positionGeneral_x', 'league', 'leagueLevel',
       'tradeId', 'regularGamesPlayed', 'regularGoals', 'regularAssists',
       'regularPenaltyMinutes', 'postseasonGamesPlayed', 'postseasonGoals',
       'postseasonAssists', 'postseasonPenaltyMinutes', 'regular_hits',
       'regular_blockedShots', 'regular_powerplayGoals', 'regular_shots',
       'regular_atoi', 'regular_pp_atoi', 'regular_pk_atoi', 'nhlPlayerId',
       'trade_date', 'positionGeneral_y', 'height_cm', 'weight_kg',
       'handedness', 'dateOfBirth', 'birthCountry', 'draftYear', 'draftRound',
       'draftOverallPick', 'contractId', 'season', 'seasonStart', 'seasonEnd',
       'aav', 'toTeamId', 'fromTeamId', 'amountOfCapRetained', 'acquiringCap'],
      dtype='object')

In [22]:
team_id_mapping.columns

Index(['teamId', 'season', 'teamName', 'teamAbbreviation',
       'conferenceAbbreviation', 'divisionName'],
      dtype='object')

In [23]:
# Merge the to team ID mapping data to the player's pre trade team
fifth_merge = pd.merge(fourth_merge, team_id_mapping, left_on=['fromTeamId','season'], right_on=['teamId','season'], how='inner')

In [24]:
fifth_merge.shape

(1473, 46)

In [25]:
# Do this again with the player's post trade team
sixth_merge = pd.merge(fifth_merge, team_id_mapping, left_on=['toTeamId','season'], right_on=['teamId','season'], how='inner', suffixes=('_pre_trade','_post_trade'))

In [26]:
sixth_merge.shape

(1470, 51)

## New formatting and merging

In [27]:
sixth_merge.columns

Index(['playerId', 'position', 'positionGeneral_x', 'league', 'leagueLevel',
       'tradeId', 'regularGamesPlayed', 'regularGoals', 'regularAssists',
       'regularPenaltyMinutes', 'postseasonGamesPlayed', 'postseasonGoals',
       'postseasonAssists', 'postseasonPenaltyMinutes', 'regular_hits',
       'regular_blockedShots', 'regular_powerplayGoals', 'regular_shots',
       'regular_atoi', 'regular_pp_atoi', 'regular_pk_atoi', 'nhlPlayerId',
       'trade_date', 'positionGeneral_y', 'height_cm', 'weight_kg',
       'handedness', 'dateOfBirth', 'birthCountry', 'draftYear', 'draftRound',
       'draftOverallPick', 'contractId', 'season', 'seasonStart', 'seasonEnd',
       'aav', 'toTeamId', 'fromTeamId', 'amountOfCapRetained', 'acquiringCap',
       'teamId_pre_trade', 'teamName_pre_trade', 'teamAbbreviation_pre_trade',
       'conferenceAbbreviation_pre_trade', 'divisionName_pre_trade',
       'teamId_post_trade', 'teamName_post_trade',
       'teamAbbreviation_post_trade', 'conferen

In [28]:
# Convert trade date column to date data type
sixth_merge['trade_date'] = pd.to_datetime(sixth_merge['trade_date'], format='%Y-%m-%d')

In [29]:
# Create a trade season column instead of just converting the seasons in the skater data
# This is because of a year mix up in the data pulled from the platform
# This column will encapsulate the season in which either a player was traded during or
# the season after a player was traded during the off season
sixth_merge['post_trade_season'] = sixth_merge['trade_date'].apply(lambda x: '24-25' if (pd.to_datetime('2024-04-01') <= x <= pd.to_datetime('2025-03-31')) else ('23-24' if pd.to_datetime('2023-04-01') <= x <= pd.to_datetime('2024-03-31') else ('22-23' if pd.to_datetime('2022-04-01') <= x <= pd.to_datetime('2023-03-31') else ('21-22' if pd.to_datetime('2021-04-01') <= x <= pd.to_datetime('2022-03-31') else ('20-21' if pd.to_datetime('2020-04-01') <= x <= pd.to_datetime('2021-03-31') else ('19-20' if pd.to_datetime('2019-04-01') <= x <= pd.to_datetime('2020-03-31') else ('18-19' if pd.to_datetime('2018-04-01') <= x <= pd.to_datetime('2019-03-31') else ('17-18' if pd.to_datetime('2017-04-01') <= x <= pd.to_datetime('2018-03-31') else ('16-17' if pd.to_datetime('2016-04-01') <= x <= pd.to_datetime('2017-03-31') else ('15-16' if pd.to_datetime('2015-04-01') <= x <= pd.to_datetime('2016-03-31') else ('14-15' if pd.to_datetime('2014-04-01') <= x <= pd.to_datetime('2015-03-31') else ('13-14' if pd.to_datetime('2013-04-01') <= x <= pd.to_datetime('2014-03-31') else ('12-13' if pd.to_datetime('2012-04-01') <= x <= pd.to_datetime('2013-03-31') else ('11-12' if pd.to_datetime('2011-04-01') <= x <= pd.to_datetime('2012-03-31') else ('10-11' if pd.to_datetime('2010-04-01') <= x <= pd.to_datetime('2011-03-31') else ('09-10' if pd.to_datetime('2009-04-01') <= x <= pd.to_datetime('2010-03-31') else ('08-09' if pd.to_datetime('2008-04-01') <= x <= pd.to_datetime('2009-03-31') else (None))))))))))))))))))

In [30]:
# Make a separate column with just the season month and day combination
sixth_merge['trade_month_day'] = pd.to_datetime(sixth_merge['trade_date']).dt.strftime('%m-%d')

In [31]:
# Create a boolean variable for if the player was traded in the off-season
sixth_merge['off-season_trade'] = np.where(sixth_merge['trade_month_day'].between('04-01', '09-30'), 1, 0)

In [32]:
sixth_merge['off-season_trade'].value_counts()

off-season_trade
0    1051
1     419
Name: count, dtype: int64

In [33]:
# Assign pre-trade year based on the trade year
sixth_merge['pre_trade_season'] = sixth_merge['post_trade_season'].apply(lambda x: '23-24' if x == '24-25' else ('22-23' if x == '23-24' else ('21-22' if x == '22-23' else ('20-21' if x == '21-22' else ('19-20' if x == '20-21' else ('18-19' if x == '19-20' else ('17-18' if x == '18-19' else ('16-17' if x == '17-18' else ('15-16' if x == '16-17' else ('14-15' if x == '15-16' else ('13-14' if x == '14-15' else ('12-13' if x == '13-14' else ('11-12' if x == '12-13' else ('10-11' if x == '11-12' else ('09-10' if x == '10-11' else ('08-09' if x == '09-10' else (None)))))))))))))))))

In [34]:
# Change data types from int32 to int64 for consistency
sixth_merge['off-season_trade'] = sixth_merge['off-season_trade'].astype('int64')

In [35]:
# filter for players traded in the off-season only
off_season_traded_players = sixth_merge[sixth_merge['off-season_trade'] == 1]

In [36]:
off_season_traded_players.shape

(419, 55)

In [37]:
# Import skater data from 08-09 season to 24-25 season
skater_data = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_Skater_Rates.csv')

In [38]:
# Read in new dataset for team standings before the offseason in which the player was traded
team_standings = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_Team_Standings.csv')

In [39]:
# Read in team level +-G data for pre trade season
PMG_team_data = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_P_M_G_Teams.csv')

In [40]:
# Read in after the season after the trades gar stats
after_trade_gar_stats = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_GAR_Stats.csv')

In [41]:
off_season_traded_players.columns

Index(['playerId', 'position', 'positionGeneral_x', 'league', 'leagueLevel',
       'tradeId', 'regularGamesPlayed', 'regularGoals', 'regularAssists',
       'regularPenaltyMinutes', 'postseasonGamesPlayed', 'postseasonGoals',
       'postseasonAssists', 'postseasonPenaltyMinutes', 'regular_hits',
       'regular_blockedShots', 'regular_powerplayGoals', 'regular_shots',
       'regular_atoi', 'regular_pp_atoi', 'regular_pk_atoi', 'nhlPlayerId',
       'trade_date', 'positionGeneral_y', 'height_cm', 'weight_kg',
       'handedness', 'dateOfBirth', 'birthCountry', 'draftYear', 'draftRound',
       'draftOverallPick', 'contractId', 'season', 'seasonStart', 'seasonEnd',
       'aav', 'toTeamId', 'fromTeamId', 'amountOfCapRetained', 'acquiringCap',
       'teamId_pre_trade', 'teamName_pre_trade', 'teamAbbreviation_pre_trade',
       'conferenceAbbreviation_pre_trade', 'divisionName_pre_trade',
       'teamId_post_trade', 'teamName_post_trade',
       'teamAbbreviation_post_trade', 'conferen

In [42]:
skater_data.columns

Index(['Player', 'EH_ID', 'API ID', 'Season', 'Team', 'Position', 'Shoots',
       'Birthday', 'Age', 'Draft Yr', 'Draft Rd', 'Draft Ov', 'GP', 'TOI',
       'G/60', 'A1/60', 'A2/60', 'Points/60', 'iSF/60', 'iFF/60', 'iCF/60',
       'ixG/60', 'Sh%', 'FSh%', 'xFSh%', 'iBLK/60', 'GIVE/60', 'TAKE/60',
       'iHF/60', 'iHA/60', 'iPENT2/60', 'iPEND2/60', 'iPENT5/60', 'iPEND5/60',
       'iPEN±/60', 'FOW/60', 'FOL/60', 'FO±/60'],
      dtype='object')

In [43]:
off_season_traded_players['teamAbbreviation_pre_trade'].unique()

array(['TBL', 'DET', 'PHI', 'NSH', 'VAN', 'CGY', 'FLA', 'CBJ', 'NJD',
       'MIN', 'LAK', 'OTT', 'WSH', 'BOS', 'STL', 'CHI', 'CAR', 'NYR',
       'SJS', 'BUF', 'COL', 'ANA', 'TOR', 'DAL', 'MTL', 'NYI', 'VGK',
       'PIT', 'ARI', 'ATL', 'EDM', 'WPG'], dtype=object)

In [44]:
skater_data['Team'].unique()

array(['COL', 'N.J', 'BOS', 'CGY', 'FLA', 'DET', 'DAL', 'WPG', 'CHI',
       'EDM', 'CBJ', 'NYI', 'WSH', 'ARI', 'MTL', 'CAR', 'VAN', 'NYR',
       'ANA', 'MIN', 'TOR', 'VGK', 'S.J', 'PIT', 'STL', 'T.B', 'OTT',
       'PHI', 'SEA', 'BUF', 'NSH', 'L.A', 'ATL', 'UTA'], dtype=object)

In [45]:
# Change team abbreviations to match that of the ones in the evolving hockey data
team_mapping = {
    'COL': 'COL',
    'NJD': 'N.J',
    'SJS': 'S.J',
    'TBL': 'T.B',
    'LAK': 'L.A',
    'MTL': 'MTL',
    'BOS': 'BOS',
    'BUF': 'BUF',
    'MIN': 'MIN',
    'WSH': 'WSH',
    'TOR': 'TOR',
    'OTT': 'OTT',
    'PHI': 'PHI',
    'PIT': 'PIT',
    'NYR': 'NYR',
    'NYI': 'NYI',
    'FLA': 'FLA',
    'CAR': 'CAR',
    'CHI': 'CHI',
    'CBJ': 'CBJ',
    'ARI': 'ARI',
    'VGK': 'VGK',
    'DAL': 'DAL',
    'EDM': 'EDM',
    'WPG': 'WPG',
    'ANA': 'ANA',
    'VAN': 'VAN',
    'STL': 'STL',
    'CGY': 'CGY',
    'NSH': 'NSH',
    'ARI': 'ARI',
    'ATL': 'ATL',
    'DET': 'DET'
}

off_season_traded_players['teamAbbreviation_pre_trade'] = off_season_traded_players['teamAbbreviation_pre_trade'].map(team_mapping)

# Do it again for the post trade team abbreviations
off_season_traded_players['teamAbbreviation_post_trade'] = off_season_traded_players['teamAbbreviation_post_trade'].map(team_mapping)

C:\Users\ajhay\AppData\Local\Temp\ipykernel_19896\1226851261.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  off_season_traded_players['teamAbbreviation_pre_trade'] = off_season_traded_players['teamAbbreviation_pre_trade'].map(team_mapping)
C:\Users\ajhay\AppData\Local\Temp\ipykernel_19896\1226851261.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  off_season_traded_players['teamAbbreviation_post_trade'] = off_season_traded_players['teamAbbreviation_post_trade'].map(team_mapping)


In [46]:
# Merge all of the hockey skater stats with the offseason players
off_season_first_merge = pd.merge(off_season_traded_players, skater_data, left_on=['nhlPlayerId','pre_trade_season','teamAbbreviation_pre_trade'], right_on=['API ID','Season','Team'], how='inner')

## Test 1

---

In [47]:
# Merge all of the hockey skater stats with the offseason players
test1 = pd.merge(off_season_traded_players, skater_data, left_on=['nhlPlayerId','pre_trade_season','teamAbbreviation_pre_trade'], right_on=['API ID','Season','Team'], how='left')

In [48]:
test1.shape

(419, 93)

In [49]:
skater_data[(skater_data['API ID'] == 8476418)]

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,...,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60
13350,Robbie Russo,ROBBIE.RUSSO,8476418,16-17,DET,D,R,1993-02-15,23,2011,...,5.17,9.91,0.43,0.22,0.0,0.0,-0.22,0.0,0.0,0.0


In [50]:
off_season_traded_players[off_season_traded_players['nhlPlayerId'] == 8476418]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,divisionName_pre_trade,teamId_post_trade,teamName_post_trade,teamAbbreviation_post_trade,conferenceAbbreviation_post_trade,divisionName_post_trade,post_trade_season,trade_month_day,off-season_trade,pre_trade_season
125,48682,RD,D,nhl,professional,538,19,0,0,2,...,Atlantic,1574,Arizona Coyotes,ARI,W,Pacific,18-19,06-24,1,17-18


In [51]:
test1[test1['Team'].isna()]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60
5,584051,C,F,nhl,professional,510,7,0,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,48682,RD,D,nhl,professional,538,19,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,813211,LD,D,nhl,professional,2341,55,1,6,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,11247,LD,D,nhl,professional,524,76,2,8,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,105652,LD,D,nhl,professional,401,68,0,12,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,32231,RD,D,nhl,professional,1446,61,5,26,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,180420,RD,D,nhl,professional,1132,38,1,5,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,692304,RD,D,nhl,professional,1406,3,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,575744,LD,D,nhl,professional,1486,82,6,21,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,730879,LD,D,nhl,professional,2092,10,1,1,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Results: Some team abreviations were not the same in evolving hockey vs the kings data so they were fixed and matched, this added back several rows. However, there are still 49 rows that are dropped due to their stats not being in the evolving hockey dataset according to the first few rows. 

---

In [52]:
off_season_first_merge.shape

(370, 93)

In [53]:
team_standings.columns

Index(['Name', 'Team', 'Season', 'GP', 'TOI', 'W/GP', 'L/GP', 'OL/GP',
       'ROW/GP', 'Points/GP', 'Points%', 'GF/GP', 'GA/GP', 'G±/GP', 'Sh%',
       'Sv%'],
      dtype='object')

In [54]:
# Merge all of the hockey teams stats with the offseason players
off_season_second_merge = pd.merge(off_season_first_merge, team_standings, left_on=['pre_trade_season','teamAbbreviation_pre_trade'], right_on=['Season','Team'], how='inner', suffixes=('_skater','_team'))

In [55]:
off_season_traded_players[['playerId','tradeId']].value_counts()

playerId  tradeId
60638     172        2
575744    1486       2
142907    235        2
610535    61         1
610355    756        1
                    ..
110272    807        1
107968    243        1
107788    39         1
107668    286        1
864629    1315       1
Name: count, Length: 416, dtype: int64

In [56]:
off_season_second_merge.shape

(370, 109)

In [57]:
PMG_team_data.columns

Index(['Name', 'Team', 'Season', 'GP', 'TOI', 'FF/60', 'FA/60', 'xGF/FF',
       'xGA/FA', 'RF/60', 'RA/60', 'QF/60', 'QA/60', 'R±/60', 'Q±/60',
       'Sh±/60', 'Sv±/60', 'G±/60'],
      dtype='object')

In [58]:
# Merge all of the hockey teams stats with the offseason players
off_season_third_merge = pd.merge(off_season_second_merge, PMG_team_data, left_on=['pre_trade_season','teamAbbreviation_pre_trade'], right_on=['Season','Team'], how='inner', suffixes=('_standings','_PMG'))

In [59]:
off_season_third_merge.shape

(370, 127)

In [60]:
off_season_third_merge[['playerId','tradeId']].value_counts()

playerId  tradeId
11128     1043       1
548210    1135       1
606851    667        1
596486    1626       1
593280    749        1
                    ..
107788    39         1
107668    286        1
107548    224        1
107087    1146       1
846660    104        1
Name: count, Length: 370, dtype: int64

In [61]:
after_trade_gar_stats.columns

Index(['Player', 'EH_ID', 'API ID', 'Season', 'Team', 'Position', 'Shoots',
       'Birthday', 'Age', 'Draft Yr', 'Draft Rd', 'Draft Ov', 'GP', 'TOI_All',
       'TOI_EV', 'TOI_PP', 'TOI_SH', 'EVO_GAR/60', 'EVD_GAR/60', 'PPO_GAR/60',
       'SHD_GAR/60', 'Take_GAR/60', 'Draw_GAR/60', 'Off_GAR/60', 'Def_GAR/60',
       'Pens_GAR/60', 'GAR/60', 'WAR/60', 'SPAR/60'],
      dtype='object')

In [62]:
off_season_third_merge.columns

Index(['playerId', 'position', 'positionGeneral_x', 'league', 'leagueLevel',
       'tradeId', 'regularGamesPlayed', 'regularGoals', 'regularAssists',
       'regularPenaltyMinutes',
       ...
       'xGA/FA', 'RF/60', 'RA/60', 'QF/60', 'QA/60', 'R±/60', 'Q±/60',
       'Sh±/60', 'Sv±/60', 'G±/60'],
      dtype='object', length=127)

In [63]:
# Merge all of the hockey teams stats with the offseason players
off_season_fourth_merge = pd.merge(off_season_third_merge, after_trade_gar_stats, left_on=['nhlPlayerId','post_trade_season','teamAbbreviation_post_trade'], right_on=['API ID','Season','Team'], how='inner')


In [64]:
off_season_fourth_merge.shape

(283, 156)

In [65]:
off_season_fourth_merge[['playerId','tradeId']].value_counts()

playerId  tradeId
11186     354        1
579951    745        1
619272    589        1
617129    777        1
613296    1304       1
                    ..
110332    35         1
119040    238        1
119158    293        1
119160    1139       1
846660    104        1
Name: count, Length: 283, dtype: int64

# Test 2

---

In [66]:
# Merge all of the hockey skater stats with the offseason players
test2 = pd.merge(off_season_third_merge, after_trade_gar_stats, left_on=['nhlPlayerId','post_trade_season','teamAbbreviation_post_trade'], right_on=['API ID','Season','Team'], how='left')

In [67]:
test2.shape

(370, 156)

In [104]:
after_trade_gar_stats[(after_trade_gar_stats['API ID'] == 8474818)]

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,...,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
8084,Jordie Benn,JORDIE.BENN,8474818,11-12,DAL,D,L,1987-07-26,24,2005,...,-0.017,0.656,0.133,-0.068,-0.079,0.459,0.065,0.408,0.071,0.132
8085,Jordie Benn,JORDIE.BENN,8474818,12-13,DAL,D,L,1987-07-26,25,2005,...,-1.240,-1.392,-0.003,0.010,-0.127,-0.119,0.006,-0.214,-0.040,-0.073
8086,Jordie Benn,JORDIE.BENN,8474818,13-14,DAL,D,L,1987-07-26,26,2005,...,1.768,0.170,0.031,-0.004,0.120,0.072,0.027,0.203,0.038,0.072
8087,Jordie Benn,JORDIE.BENN,8474818,14-15,DAL,D,L,1987-07-26,27,2005,...,0.128,-0.189,-0.029,-0.001,0.031,0.163,-0.030,0.153,0.029,0.056
8088,Jordie Benn,JORDIE.BENN,8474818,15-16,DAL,D,L,1987-07-26,28,2005,...,2.864,0.267,0.042,-0.016,0.270,0.012,0.026,0.272,0.052,0.101
8089,Jordie Benn,JORDIE.BENN,8474818,16-17,DAL,D,L,1987-07-26,29,2005,...,-1.318,-0.117,0.005,0.005,-0.245,-0.202,0.009,-0.386,-0.073,-0.141
8090,Jordie Benn,JORDIE.BENN,8474818,16-17,MTL,D,L,1987-07-26,29,2005,...,0.000,-0.348,0.075,0.040,-0.201,0.325,0.115,0.260,0.049,0.095
8091,Jordie Benn,JORDIE.BENN,8474818,17-18,MTL,D,L,1987-07-26,30,2005,...,-0.030,-0.681,0.040,-0.032,-0.280,0.091,0.008,-0.146,-0.028,-0.054
8092,Jordie Benn,JORDIE.BENN,8474818,18-19,MTL,D,L,1987-07-26,31,2005,...,-0.441,0.662,0.016,-0.004,0.230,0.235,0.013,0.435,0.081,0.153
8093,Jordie Benn,JORDIE.BENN,8474818,19-20,VAN,D,L,1987-07-26,32,2005,...,0.433,0.570,0.034,0.048,-0.202,-0.195,0.082,-0.283,-0.050,-0.097


In [105]:
off_season_third_merge[off_season_third_merge['nhlPlayerId'] == 8474818]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,xGA/FA,RF/60,RA/60,QF/60,QA/60,R±/60,Q±/60,Sh±/60,Sv±/60,G±/60
15,52195,D,D,nhl,professional,211,39,1,9,9,...,0.0011,-0.06,0.33,-0.11,0.05,-0.4,-0.16,0.07,-0.05,-0.54


In [70]:
test2[test2['Team_y'].isna()]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
5,730758,C,F,nhl,professional,1065,45,3,3,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,41813,C,F,nhl,professional,435,25,1,4,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,52195,D,D,nhl,professional,211,39,1,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,450148,RW,F,nhl,professional,1014,3,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,596486,C,F,nhl,professional,1626,5,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,490369,C,F,nhl,professional,875,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,11128,LW,F,nhl,professional,1043,2,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,51953,LW,F,nhl,professional,1132,4,1,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,153674,C,F,nhl,professional,4331,79,13,11,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


results: According to the first few rows, this merge faces the same issues as the first test did. There is no data for the corresponding season for a specific player. However, this was discovered to be for a few different reasons. Either there was no data listed for a player in that year or that player went to a seperate league such as nhlPlayerID 8468162 or Jerred Smithson. He was traded to the Oilers from the Panthers but went to the toronto Marlies shotrly before the season began. Think of ways to observe transactions inside of a season? Seperate them with dates somehow?

---

## Now, let's format some of the data and merge all of the evolving hockey data for the pre-trade stats.

In [71]:
# Convert trade date column to date data type
sixth_merge['trade_date'] = pd.to_datetime(sixth_merge['trade_date'], format='%Y-%m-%d')

In [72]:
# Create a trade season column instead of just converting the seasons in the skater data
# This is because of a year mix up in the data pulled from the platform
# This column will encapsulate the season in which either a player was traded during or
# the season after a player was traded during the off season
sixth_merge['traded_season'] = sixth_merge['trade_date'].apply(lambda x: '24-25' if (pd.to_datetime('2024-04-01') <= x <= pd.to_datetime('2025-03-31')) else ('23-24' if pd.to_datetime('2023-04-01') <= x <= pd.to_datetime('2024-03-31') else ('22-23' if pd.to_datetime('2022-04-01') <= x <= pd.to_datetime('2023-03-31') else ('21-22' if pd.to_datetime('2021-04-01') <= x <= pd.to_datetime('2022-03-31') else ('20-21' if pd.to_datetime('2020-04-01') <= x <= pd.to_datetime('2021-03-31') else ('19-20' if pd.to_datetime('2019-04-01') <= x <= pd.to_datetime('2020-03-31') else ('18-19' if pd.to_datetime('2018-04-01') <= x <= pd.to_datetime('2019-03-31') else ('17-18' if pd.to_datetime('2017-04-01') <= x <= pd.to_datetime('2018-03-31') else ('16-17' if pd.to_datetime('2016-04-01') <= x <= pd.to_datetime('2017-03-31') else ('15-16' if pd.to_datetime('2015-04-01') <= x <= pd.to_datetime('2016-03-31') else ('14-15' if pd.to_datetime('2014-04-01') <= x <= pd.to_datetime('2015-03-31') else ('13-14' if pd.to_datetime('2013-04-01') <= x <= pd.to_datetime('2014-03-31') else ('12-13' if pd.to_datetime('2012-04-01') <= x <= pd.to_datetime('2013-03-31') else ('11-12' if pd.to_datetime('2011-04-01') <= x <= pd.to_datetime('2012-03-31') else ('10-11' if pd.to_datetime('2010-04-01') <= x <= pd.to_datetime('2011-03-31') else ('09-10' if pd.to_datetime('2009-04-01') <= x <= pd.to_datetime('2010-03-31') else ('08-09' if pd.to_datetime('2008-04-01') <= x <= pd.to_datetime('2009-03-31') else (None))))))))))))))))))

In [73]:
# Make a separate column with just the season month and day combination
sixth_merge['trade_month_day'] = pd.to_datetime(sixth_merge['trade_date']).dt.strftime('%m-%d')

In [74]:
# Create a boolean variable for if the player was traded in the off-season
sixth_merge['off-season_trade'] = np.where(sixth_merge['trade_month_day'].between('04-01', '09-30'), 1, 0)

In [75]:
sixth_merge['off-season_trade'].value_counts()

off-season_trade
0    1051
1     419
Name: count, dtype: int64

In [76]:
# Assign pre-trade year based on the trade year
sixth_merge['pre_trade_season'] = sixth_merge['traded_season'].apply(lambda x: '23-24' if x == '24-25' else ('22-23' if x == '23-24' else ('21-22' if x == '22-23' else ('20-21' if x == '21-22' else ('19-20' if x == '20-21' else ('18-19' if x == '19-20' else ('17-18' if x == '18-19' else ('16-17' if x == '17-18' else ('15-16' if x == '16-17' else ('14-15' if x == '15-16' else ('13-14' if x == '14-15' else ('12-13' if x == '13-14' else ('11-12' if x == '12-13' else ('10-11' if x == '11-12' else ('09-10' if x == '10-11' else ('08-09' if x == '09-10' else (None)))))))))))))))))

In [77]:
# Change data types from int32 to int64 for consistency
sixth_merge['off-season_trade'] = sixth_merge['off-season_trade'].astype('int64')

In [78]:
# filter for players traded in the off-season only
off_season_traded_players = sixth_merge[sixth_merge['off-season_trade'] == 1]

In [79]:
off_season_traded_players.shape

(419, 56)

In [80]:
# Import skater data from 08-09 season to 24-25 season
skater_data = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_Skater_Rates.csv')

In [81]:
skater_data.columns

Index(['Player', 'EH_ID', 'API ID', 'Season', 'Team', 'Position', 'Shoots',
       'Birthday', 'Age', 'Draft Yr', 'Draft Rd', 'Draft Ov', 'GP', 'TOI',
       'G/60', 'A1/60', 'A2/60', 'Points/60', 'iSF/60', 'iFF/60', 'iCF/60',
       'ixG/60', 'Sh%', 'FSh%', 'xFSh%', 'iBLK/60', 'GIVE/60', 'TAKE/60',
       'iHF/60', 'iHA/60', 'iPENT2/60', 'iPEND2/60', 'iPENT5/60', 'iPEND5/60',
       'iPEN±/60', 'FOW/60', 'FOL/60', 'FO±/60'],
      dtype='object')

In [82]:
sixth_merge.columns

Index(['playerId', 'position', 'positionGeneral_x', 'league', 'leagueLevel',
       'tradeId', 'regularGamesPlayed', 'regularGoals', 'regularAssists',
       'regularPenaltyMinutes', 'postseasonGamesPlayed', 'postseasonGoals',
       'postseasonAssists', 'postseasonPenaltyMinutes', 'regular_hits',
       'regular_blockedShots', 'regular_powerplayGoals', 'regular_shots',
       'regular_atoi', 'regular_pp_atoi', 'regular_pk_atoi', 'nhlPlayerId',
       'trade_date', 'positionGeneral_y', 'height_cm', 'weight_kg',
       'handedness', 'dateOfBirth', 'birthCountry', 'draftYear', 'draftRound',
       'draftOverallPick', 'contractId', 'season', 'seasonStart', 'seasonEnd',
       'aav', 'toTeamId', 'fromTeamId', 'amountOfCapRetained', 'acquiringCap',
       'teamId_pre_trade', 'teamName_pre_trade', 'teamAbbreviation_pre_trade',
       'conferenceAbbreviation_pre_trade', 'divisionName_pre_trade',
       'teamId_post_trade', 'teamName_post_trade',
       'teamAbbreviation_post_trade', 'conferen

In [83]:
# Assign pre-trade stats based on the trade year
pre_season_stats_merged = pd.merge(off_season_traded_players, skater_data, left_on=['nhlPlayerId','pre_trade_season','draftYear','draftRound'], right_on=['API ID','Season','Draft Yr','Draft Rd'], how='inner')

In [84]:
# Read in new dataset for team standings before the offseason in which the player was traded
team_standings = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_Team_Standings.csv')

In [85]:
# Assign pre-trade stats from players old team based on the trade year
all_pre_season_stats = pd.merge(pre_season_stats_merged, team_standings, left_on=['Team','pre_trade_season'], right_on=['Team','Season'], how='inner',suffixes=('indv','team'))

In [86]:
# Read in team level +-G data for pre trade season
PMG_team_data = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_P_M_G_Teams.csv')

In [87]:
# Assign pre-trade stats from players old team based on the trade year
all_pre_season_stats = pd.merge(all_pre_season_stats, PMG_team_data, left_on=['Team','pre_trade_season'], right_on=['Team','Season'], how='inner',suffixes=('_1','_2'))

In [88]:
all_pre_season_stats.shape

(384, 126)

In [89]:
# Remove all duplicate columns from previous merges that end in _y
'''all_pre_season_stats.drop(columns=['positionGeneral_y','Player_y', 'EH_ID_y', 'API ID_y',
       'Season_y', 'Position_y', 'Shoots_y', 'Birthday_y', 'Age_y',
       'Draft Yr_y', 'Draft Rd_y', 'Draft Ov_y', 'GP_y', 'TOI_y'], inplace=True)'''

"all_pre_season_stats.drop(columns=['positionGeneral_y','Player_y', 'EH_ID_y', 'API ID_y',\n       'Season_y', 'Position_y', 'Shoots_y', 'Birthday_y', 'Age_y',\n       'Draft Yr_y', 'Draft Rd_y', 'Draft Ov_y', 'GP_y', 'TOI_y'], inplace=True)"

In [90]:
# Get rid of all _x suffixes from the columns
# all_pre_season_stats.columns = [col.replace('_x','') for col in all_pre_season_stats.columns]

In [91]:
all_pre_season_stats.shape

(384, 126)

In [92]:
# Read in after the season after the trades gar stats
after_trade_gar_stats = pd.read_csv(r'../Evolving-Hockey/NHL_08_24_GAR_Stats.csv')

In [93]:
after_trade_gar_stats.columns

Index(['Player', 'EH_ID', 'API ID', 'Season', 'Team', 'Position', 'Shoots',
       'Birthday', 'Age', 'Draft Yr', 'Draft Rd', 'Draft Ov', 'GP', 'TOI_All',
       'TOI_EV', 'TOI_PP', 'TOI_SH', 'EVO_GAR/60', 'EVD_GAR/60', 'PPO_GAR/60',
       'SHD_GAR/60', 'Take_GAR/60', 'Draw_GAR/60', 'Off_GAR/60', 'Def_GAR/60',
       'Pens_GAR/60', 'GAR/60', 'WAR/60', 'SPAR/60'],
      dtype='object')

In [94]:
# merge the pre season stats with the after trade gar stats
final_data = pd.merge(all_pre_season_stats, after_trade_gar_stats, left_on=['nhlPlayerId','traded_season','EH_ID'], right_on=['API ID','Season','EH_ID'], how='inner')

In [95]:
final_data.shape

(335, 154)

In [96]:
final_data[['nhlPlayerId','tradeId','traded_season']].value_counts()

nhlPlayerId  tradeId  traded_season
8475283      867      13-14            4
8477406      235      21-22            4
8475233      172      21-22            4
8467925      749      13-14            3
8474688      322      21-22            3
                                      ..
8474001      1931     13-14            1
8474037      1246     12-13            1
8474038      870      11-12            1
8474056      423      19-20            1
8482125      2340     24-25            1
Name: count, Length: 290, dtype: int64

In [97]:
final_data.drop_duplicates(inplace=True)

In [98]:
final_data[(final_data['nhlPlayerId'] == 8475283) & (final_data['tradeId'] == 867)]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
314,634157,LW,F,nhl,professional,867,9,2,0,19,...,0.00,0.0,0.149,-0.138,0.105,-0.230,0.011,-0.113,-0.021,-0.040
315,634157,LW,F,nhl,professional,867,9,2,0,19,...,-0.21,0.0,0.149,-0.138,0.706,0.605,0.011,1.321,0.250,0.468
316,634157,LW,F,nhl,professional,867,9,2,0,19,...,0.00,0.0,0.149,-0.138,0.105,-0.230,0.011,-0.113,-0.021,-0.040
317,634157,LW,F,nhl,professional,867,9,2,0,19,...,-0.21,0.0,0.149,-0.138,0.706,0.605,0.011,1.321,0.250,0.468


In [99]:
final_data[(final_data['nhlPlayerId'] == 8467925) & (final_data['tradeId'] == 749)]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
235,740013,RW,F,nhl,professional,749,37,0,4,23,...,-0.114,-0.271,0.012,-0.044,-0.109,0.004,-0.032,-0.104,-0.02,-0.037
236,740013,RW,F,nhl,professional,749,37,0,4,23,...,-0.114,-0.271,0.012,-0.044,-0.109,0.004,-0.032,-0.104,-0.02,-0.037
237,740013,RW,F,nhl,professional,749,37,0,4,23,...,-0.114,-0.271,0.012,-0.044,-0.109,0.004,-0.032,-0.104,-0.02,-0.037


In [100]:
final_data[(final_data['nhlPlayerId'] == 8477986) & (final_data['tradeId'] == 271)]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
260,126984,RD,D,nhl,professional,271,50,7,11,40,...,1.133,0.167,-0.03,0.017,0.307,-0.079,-0.014,0.208,0.037,0.069
261,126984,RD,D,nhl,professional,271,50,7,11,40,...,1.133,0.167,-0.03,0.017,0.307,-0.079,-0.014,0.208,0.037,0.069


In [101]:
third_merge[(third_merge['nhlPlayerId'] == 8477986) & (third_merge['tradeId'] == 271)]

,playerId,position,positionGeneral_x,league,leagueLevel,tradeId,regularGamesPlayed,regularGoals,regularAssists,regularPenaltyMinutes,...,trade_date,positionGeneral_y,height_cm,weight_kg,handedness,dateOfBirth,birthCountry,draftYear,draftRound,draftOverallPick
1109,126984,RD,D,nhl,professional,271,50,7,11,40,...,2021-04-10,D,182.88,90.0,R,1994-04-11,Canada,2014.0,2.0,55.0
